In [13]:
import matplotlib.pyplot as plt

from constants import *
from download import download_monthly_klines as download
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

if __name__ == '__main__':
    i = 0
    # download(trading_type=TRADING_TYPE[0], symbols=[TICKERS[i] +
    #          BASE],intervals=['1m'], years=[2020, 2021], months=list(range(1, 13)))
    all_files = glob.glob(f'../data/{TICKERS[i] + BASE}/15m/*.csv')
    # print(all_files)
    # pd.read_csv(f'../data/{TICKERS[i] + BASE}/15m')
    li = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None,
                         names=["Open time", "Open", "High", "Low", "Close", "Volume", "Close time",
                                "Quote asset volume", "Number of trades", "Taker buy base asset volume",
                                "Taker buy quote asset volume", "Ignore"])
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    # print(frame)


In [14]:
frame['is_green'] = frame['Open'] < frame['Close']
frame.drop(['Open time', 'Close time', 'Ignore'], axis='columns', inplace=True)

In [15]:
frame

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,is_green
0,41461.84,41566.05,41120.00,41487.99,1179.035106,4.875550e+07,26973,570.613922,2.360490e+07,True
1,41487.99,41659.88,41421.89,41624.67,420.119954,1.746358e+07,14347,189.467821,7.876374e+06,True
2,41631.60,41708.56,41543.25,41686.38,441.926215,1.839481e+07,14183,224.671731,9.352563e+06,True
3,41686.39,41772.87,41652.24,41737.37,503.409429,2.100226e+07,14264,220.942165,9.218120e+06,True
4,41734.66,41920.00,41653.21,41860.98,619.302471,2.589235e+07,30924,325.907532,1.362867e+07,True
...,...,...,...,...,...,...,...,...,...,...
75688,57367.82,57386.64,57167.92,57267.91,183.601300,1.051553e+07,7549,91.177860,5.222311e+06,False
75689,57267.90,57424.23,57003.67,57085.48,281.676240,1.611701e+07,9304,143.213180,8.197539e+06,False
75690,57085.48,57360.49,57084.92,57247.46,182.012370,1.041692e+07,8799,97.284000,5.568400e+06,True
75691,57247.46,57276.69,57005.10,57121.35,182.116610,1.039671e+07,8309,91.447110,5.220287e+06,False


In [16]:
frame_dynamic = frame[['Open', 'Close']]
frame_dynamic.iloc[:10]

,Open,Close
0,41461.84,41487.99
1,41487.99,41624.67
2,41631.60,41686.38
3,41686.39,41737.37
4,41734.66,41860.98
5,41860.99,41733.17
6,41733.17,41739.79
7,41739.80,41707.22
8,41707.22,41634.32
9,41634.32,41710.00


In [222]:
# [dollar, asset]

PERIOD_TESTED = 3
PREDICTION_PERIOD = 2
FEE = 0.0
actions = []  # 0 - sell
state = []
cash = np.log(1)
asset = 0
portfolio = np.log(1)
state.append([portfolio, portfolio])
tmp_states = np.zeros((2, PERIOD_TESTED, PREDICTION_PERIOD))


def action(state, current, next, fee=FEE):
    fee = np.log(1 - fee)
    if len(next) > 1:
        current = current * int(bool(state)) + max(
            (action(state=1, current=next.iloc[1], next=next.iloc[1:]) - (fee * int(bool(state)))),
            (action(state=0, current=next.iloc[1], next=next.iloc[1:]) + (fee * (1 - int(bool(state)))))
        )
    else:
        current = (current - fee) * int(bool(state))
    return current


def optimal(state, frame):
    frame = np.log(frame[1:])
    while len(state) < len(frame):
        state.append(
            [action(state=0, current=frame.iloc[len(state) - 1],
                    next=frame[len(state) - 1:len(state) + PREDICTION_PERIOD]),
             action(state=1, current=frame.iloc[len(state) - 1],
                    next=frame[len(state) - 1:len(state) + PREDICTION_PERIOD])])


optimal(state, (frame_dynamic.iloc[:PERIOD_TESTED]['Close'].pct_change().fillna(0) + 1))


def iterative(state, frame, fee=FEE):
    frame = np.log(frame[1:])
    fee = np.log(1 - fee)

    for i in range(PREDICTION_PERIOD):
        for j in range(len(frame) - 1, -1, -1):
            for x in range(2):
                if i:
                    print('xd')
                    tmp_states[1 - x, j, PREDICTION_PERIOD - i - 1] = (frame.iloc[j] * x) + max(
                        tmp_states[1, j + 1, PREDICTION_PERIOD - i - 2] - (fee * x),
                        tmp_states[0, j + 1, PREDICTION_PERIOD - i - 2] + (fee * (1 - x)))
                else:
                    tmp_states[1 - x, j, PREDICTION_PERIOD - i - 1] = (frame.iloc[j] - fee) * x

        # current = current * int(bool(state)) + max(


#            (action(state=1, current=next.iloc[1], next=next.iloc[1:]) - (fee * int(bool(state)))),
#            (action(state=0, current=next.iloc[1], next=next.iloc[1:]) + (fee * (1 - int(bool(state)))))
#        )

iterative([np.log(1), np.log(1)], (frame_dynamic.iloc[:PERIOD_TESTED]['Close'].pct_change().fillna(0) + 1))
print(tmp_states[:, 1:-1, -1])
print(state[1:])
# tmp_states

xd
xd
xd
xd
[[0.00148144]
 [0.        ]]
[[0.0014814364069594404, 0.004770468985485549]]


In [193]:
# df = pd.DataFrame(state)
# df['asset_value'] = df['asset'] * frame_dynamic['Close']
# df['current_value'] = df['cash'] + df['asset_value']
# df['cash_copy'] = df['cash']
# # df['asset'] = df.asset.shift(1, fill_value=0.0)
# # print(df)
# # df['cash'].replace(to_replace=0.0, method='ffill')
# for index in df.index:
#     if df.loc[index, 'cash'] == 0:
#         df.loc[index, 'cash'] = df.loc[index - 1, 'cash']
# df
s = np.log((frame_dynamic.iloc[:PERIOD_TESTED]['Close'].pct_change().fillna(0) + 1))

s = s[2:12]
s
sum(s[s > 0])

0.0056611077737955115

In [ ]:
df['cash'].plot()
df['current_value'].plot()
data = pd.DataFrame((frame_dynamic['Close'][:PERIOD_TESTED - 2] / frame_dynamic['Close'][0]) * 100)
# data['Close'].plot()
plt.legend()

In [ ]:
df['pct_change'] = df['cash'].pct_change()
df[df['pct_change'] != 0]

